# 🤖 Importar librerías necesarias
En esta sección se importan todas las librerías requeridas para el análisis y la inferencia: pandas, numpy, matplotlib, seaborn, scikit-learn, streamlit y holidays.

In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

# 🤖 Cargar archivo ventas_2025_inferencia en inferencia_df
En esta sección se carga el archivo de inferencia y se muestran las primeras filas para verificar la carga correcta.

In [5]:
# Cargar archivo de inferencia
inferencia_df = pd.read_csv('../data/raw/inferencia/ventas_2025_inferencia.csv')

# Mostrar las primeras filas para verificar la carga
display(inferencia_df.head())

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [6]:
# 🤖 Procesamiento y transformación de inferencia_df para inferencia
# Replicando el proceso del notebook de entrenamiento, sin dependencias externas

# 1. Manejo de valores faltantes
inferencia_df = inferencia_df.fillna(inferencia_df.mean(numeric_only=True))

# 2. Convertir columna 'fecha' a datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 3. Ingeniería de variables temporales
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['mes_nombre'] = inferencia_df['fecha'].dt.month_name()
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek  # 0=Lunes, 6=Domingo
inferencia_df['nombre_dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['dia_semana_num'] = inferencia_df['dia_semana']

# 4. Variables de calendario y eventos
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin([5, 6])
import holidays
es_holidays = holidays.Spain(years=inferencia_df['año'].unique())
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)
def es_black_friday(fecha):
    if fecha.month == 11 and fecha.weekday() == 4:
        if 22 <= fecha.day <= 28:
            return True
    return False
inferencia_df['es_black_friday'] = inferencia_df['fecha'].apply(es_black_friday)
def es_cyber_monday(fecha):
    if fecha.month == 11 and fecha.weekday() == 0:
        if 25 <= fecha.day <= 31:
            black_friday = fecha - pd.Timedelta(days=3)
            if es_black_friday(black_friday):
                return True
    return False
inferencia_df['es_cyber_monday'] = inferencia_df['fecha'].apply(es_cyber_monday)
inferencia_df['es_navidad'] = (inferencia_df['fecha'].dt.month == 12) & (inferencia_df['fecha'].dt.day == 25)
inferencia_df['es_ano_nuevo'] = (inferencia_df['fecha'].dt.month == 1) & (inferencia_df['fecha'].dt.day == 1)
inferencia_df['es_reyes'] = (inferencia_df['fecha'].dt.month == 1) & (inferencia_df['fecha'].dt.day == 6)
inferencia_df['es_semana_santa'] = inferencia_df['fecha'].isin([d for d in es_holidays if 'Santo' in es_holidays[d]])
inferencia_df['es_primer_dia_mes'] = inferencia_df['dia_mes'] == 1
inferencia_df['es_ultimo_dia_mes'] = inferencia_df['fecha'] == inferencia_df['fecha'] + pd.offsets.MonthEnd(0)

# 5. Crear lags y media móvil de unidades_vendidas (lag1 a lag7 y media móvil 7d)
def crear_lags_y_media(df, col_lag, lags, media_window, grupo):
    df = df.sort_values(['año', 'fecha']).copy()
    for lag in range(1, lags+1):
        df[f'{col_lag}_lag{lag}'] = df.groupby(grupo)[col_lag].shift(lag)
    df[f'{col_lag}_media_movil_{media_window}d'] = df.groupby(grupo)[col_lag].transform(lambda x: x.rolling(media_window).mean())
    return df
inferencia_df = crear_lags_y_media(inferencia_df, 'unidades_vendidas', lags=7, media_window=7, grupo='año')
inferencia_df = inferencia_df.dropna(subset=[f'unidades_vendidas_lag{i}' for i in range(1,8)] + ['unidades_vendidas_media_movil_7d'])

# 6. Variable de descuento porcentual respecto al precio base
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# 7. Variable precio_competencia y ratio_precio
competidores = ['Amazon', 'Decathlon', 'Deporvillage']
inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
inferencia_df = inferencia_df.drop(columns=competidores)

# 8. Copias de nombre, categoria y subcategoria con sufijo _h y one-hot encoding
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']
inferencia_df = pd.get_dummies(inferencia_df, columns=['nombre_h', 'categoria_h', 'subcategoria_h'])

# 9. Filtrar registros de noviembre (eliminar octubre)
inferencia_df = inferencia_df[inferencia_df['mes'] == 11]

# 10. Guardar el dataframe transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)

# Mostrar las primeras filas del dataframe transformado
display(inferencia_df.head())

C:\Users\Alexi\AppData\Local\Temp\ipykernel_19316\520070745.py:25: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(es_holidays)
C:\Users\Alexi\AppData\Local\Temp\ipykernel_19316\520070745.py:43: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_semana_santa'] = inferencia_df['fecha'].isin([d for d in es_holidays if 'Santo' in es_holidays[d]])


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,4.672619,115.00,554.357798,...,False,False,False,False,False,False,False,False,True,False
169,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,4.672619,135.00,554.357798,...,False,False,False,False,False,False,False,False,True,False
170,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,4.672619,86.39,554.357798,...,False,False,False,False,False,False,False,False,True,False
171,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,4.672619,74.09,554.357798,...,False,False,False,False,False,False,False,False,True,False
172,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,4.672619,34.76,554.357798,...,False,False,False,False,False,False,False,True,False,False
